In [ ]:
!pip install pandas numpy


Import Libaries and load dataset

In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
import pandas as pd
import numpy as np
import random


Set Parameters

In [ ]:
POPULATION_SIZE = 10
NUM_GENERATIONS = 3
MUTATION_RATE = 0.01
CHROMOSOME_LENGTH = len(demand_df)



In [ ]:
valid_sources_per_node = topology_df.groupby('node_id')['source_id'].apply(list).to_dict()

Chromosome Encoding

In [ ]:
def initialize_population():
    population = []
    for _ in range(POPULATION_SIZE):
        chromosome = []
        for i, row in demand_df.iterrows():
            node_id = row['node_id']
            sources = valid_sources_per_node.get(node_id, list(source_df['source_id']))
            chromosome.append(random.choice(sources))
        population.append(chromosome)
    return population





In [ ]:
def initialize_population(pop_size, gene_pool):
    return [np.random.choice(gene_pool, CHROMOSOME_LENGTH).tolist() for _ in range(pop_size)]


Fitness Function

In [ ]:
def evaluate_fitness(chromosome):
    total_cost = 0
    penalty = 0
    source_capacity_used = {}

    for i, source_id in enumerate(chromosome):
        node_id = demand_df.iloc[i]['node_id']
        demand = demand_df.iloc[i]['demand_kw']

        try:
            cost_per_kw = source_df[source_df['source_id'] == source_id]['cost_per_kw'].values[0]
            capacity = source_df[source_df['source_id'] == source_id]['capacity_kw'].values[0]
        except:
            penalty += 10000
            continue


        try:
            row = topology_df[(topology_df['source_id'] == source_id) & (topology_df['node_id'] == node_id)].iloc[0]
            loss = row['loss_coefficient']
            distance = row['distance_km']
        except:
            penalty += 5000
            continue

        actual_demand = demand * (1 + loss * distance)

        source_capacity_used[source_id] = source_capacity_used.get(source_id, 0) + actual_demand
        if source_capacity_used[source_id] > capacity:
            penalty += 1000  # Overuse

        total_cost += actual_demand * cost_per_kw

    return total_cost + penalty


Selection – Tournament

In [ ]:

def tournament_selection(population, fitness_scores, k=3):
    selected = []
    for _ in range(len(population)):
        contenders = random.sample(list(zip(population, fitness_scores)), k)
        winner = min(contenders, key=lambda x: x[1])[0]
        selected.append(winner)
    return selected




Crossover

In [ ]:

def crossover(p1, p2):
    point = random.randint(1, CHROMOSOME_LENGTH - 1)
    return p1[:point] + p2[point:]




Mutation

In [ ]:

def mutate(chromosome):
    for i in range(CHROMOSOME_LENGTH):
        if random.random() < MUTATION_RATE:
            node_id = demand_df.iloc[i]['node_id']
            sources = valid_sources_per_node.get(node_id, list(source_df['source_id']))
            chromosome[i] = random.choice(sources)
    return chromosome

Genetic Algorithm

In [ ]:
def run_ga():
    population = initialize_population()
    for gen in range(NUM_GENERATIONS):
        fitness_scores = [evaluate_fitness(ind) for ind in population]
        best_fitness = min(fitness_scores)
        avg_fitness = sum(fitness_scores) / len(fitness_scores)
        print(f"Generation {gen+1}")
        print(f"  Best Fitness: {best_fitness:.2f} | Avg Fitness: {avg_fitness:.2f}")

        selected = tournament_selection(population, fitness_scores)
        children = []

        for i in range(0, POPULATION_SIZE, 2):
            if i+1 < POPULATION_SIZE:
                child1 = crossover(selected[i], selected[i+1])
                child2 = crossover(selected[i+1], selected[i])
                child1 = mutate(child1)
                child2 = mutate(child2)
                children.extend([child1, child2])

        population = children


    final_fitness = [evaluate_fitness(ind) for ind in population]
    best_idx = np.argmin(final_fitness)
    print("Final Best Fitness:", final_fitness[best_idx])
    return population[best_idx]


In [ ]:

    final_fitness = [evaluate_fitness(ind) for ind in population]
    best_idx = np.argmin(final_fitness)
    print("Final Best Fitness:", final_fitness[best_idx])
    return population[best_idx]


best_solution = run_ga()

Generation 1
  Best Fitness: 998659.58 | Avg Fitness: 1000575.59
Generation 2
  Best Fitness: 998767.66 | Avg Fitness: 999855.18
Generation 3
  Best Fitness: 998871.88 | Avg Fitness: 999675.09
Final Best Fitness: 998841.0147609476
